In [1]:
import pandas as pd
import numpy as np

# Actual Order 반영 이전

### SOP

In [2]:
def SOP(aom, sale_forecast, month, on_hand):
  #aom actual order의 마지막 month
  def inv_prev(i):
    try:
      return inv[i-1]
    except:
      return on_hand

  oper_plan_rd = [int(sum(sale_forecast)/month)]*month
  if aom>1:
    sale_forecast = [sale_forecast[i] - 10 if i >= aom else sale_forecast[i] for i in range(month)]
  else:
    sale_forecast = [sale_forecast[i] if i >= aom else sale_forecast[i] for i in range(month)]

  fix_plan_quan = int(sum(sale_forecast[aom:])/(month-aom))
  print(fix_plan_quan)
  oper_plan = [fix_plan_quan if i >= aom else oper_plan_rd[i] for i in range(month)]

  inv = []
  for i in range(month):
    invw = inv_prev(i) + oper_plan[i] - sale_forecast[i]
    inv.append(invw)

  df = pd.DataFrame({'Sales Forecast(in units)': sale_forecast, 
                     'Actual_Order(in units)': order, 'Operations Plan(in units)': oper_plan, 
                     'Inventory Plan(in units)': inv}, index=range(1, month + 1))  
  return {'oper_plan':oper_plan,'display': df.T}

In [3]:
sale_forecast = [70,80,120,130,60,80,90,100,110,80,130,150]
month = 12
on_hand = 0
order = [0]*month

actual_order = []
aom = len(actual_order)
order = [actual_order[i] if i<=aom-1 else order[i] for i in range(month)]

In [4]:
SOP_008 = SOP(aom, sale_forecast, month, on_hand)
SOP_008['display']

100


,1,2,3,4,5,6,7,8,9,10,11,12
Sales Forecast(in units),70,80,120,130,60,80,90,100,110,80,130,150
Actual_Order(in units),0,0,0,0,0,0,0,0,0,0,0,0
Operations Plan(in units),100,100,100,100,100,100,100,100,100,100,100,100
Inventory Plan(in units),30,50,30,0,40,60,70,70,60,80,50,0


###MPS

In [5]:
ss = 30 # safety stock
ls = 300 # lot size
month = 12
on_hand = 0 # on_hand
forecast = SOP_008['oper_plan'] # SOP의 operations plan
orders= [75,120,95,60,95,80,100,110,90,95,110,130]
pab = []
mps = [0]*month
atp = [0]*month

def pab_prev(i):
  try:
    return pab[i-1]
  except:
    return on_hand

def MPS(ss,ls,month,on_hand,forecast,orders,pab,mps,atp):

  for i in range(month):
    if pab_prev(i) - max(forecast[i],orders[i]) <= ss:
      mps[i] = ls
    pabw = pab_prev(i) + mps[i] - max(forecast[i],orders[i])
    pab.append(pabw)

  indices = [i for i, x in enumerate(mps) if x == ls]

  for i in range(len(indices)):
    atpm = ls
    if i==1: # on-hand 고려
      for j in range(0,indices[i]):
        atpm = atpm - orders[j]
      atpm = atpm + on_hand
      
      atp[0] = atpm
    else:
      for j in range(indices[i-1],indices[i]):
        atpm = atpm - orders[j]
      atp[indices[i-1]] = atpm

  atpm = ls
  atpm = atpm - orders[-1]
  atp[-1] = atpm

  df = pd.DataFrame({'Forecast': forecast, 'Orders': orders, 
                     'Projected available balance': pab, 'Available-to-promise': atp, 
                     'MPS': mps}, index=range(1, month + 1))
  return {'mps':mps,'display': df.T}

MPS_008 = MPS(ss,ls,month,on_hand,forecast,orders,pab,mps,atp)
MPS_008['display']

,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,100,100,100,100,100,100,100,100,100,100,100,100
Orders,75,120,95,60,95,80,100,110,90,95,110,130
Projected available balance,200,80,280,180,80,280,180,70,270,170,60,230
Available-to-promise,105,0,50,0,0,10,0,0,5,0,0,170
MPS,300,0,300,0,0,300,0,0,300,0,0,300


### MRP

In [6]:
def MPS_to_GR(MPS, BOM):
  level1 = BOM['level1']
  GR_1 = []
  for i in level1:
    GR_1.append([i[0],[j*i[2] for j in MPS]])
  return GR_1

def POR_to_GR(POR_dict, BOM, level = 2):
  #POR_dict = ['0810', [0, 600, ... POR 값들],]
  level = BOM[f'level{level}']
  GR_next = []
  for i in level:
    key = i[1]
    POR = POR_dict[key]
    GR_next.append([i[0],[j*i[2] for j in POR]])
  return GR_next

def initialize_PAB(SR,PR,GR,PAB,I_0):
  for i in range(len(PAB)):
    if i == 0:
      PAB[i] = I_0 + SR[i] + PR[i] - GR[i]
    else:
      PAB[i] = PAB[i-1] + SR[i] + PR[i] - GR[i]
  return PAB

def update_PR(SS,SR,PR,GR,POR,PAB,I_0,LT,Lot):
  for i in range(len(PAB)):
    if PAB[i]>=SS:
      pass
    else:
      PR[i]+=Lot
      POR[i-LT]+=Lot
      PAB = initialize_PAB(SR,PR,GR,PAB,I_0)
  return PR, POR, PAB

def dataframe_return(month, GR, SR, PAB, PR, POR):
    dt = pd.DataFrame({
        'Gross requirements': GR,
        'Scheduled receipts': SR,
        'Projected available balance': PAB,
        'Planned receipts': PR,
        'Planned order releases': POR
    },index = [i for i in range(1,month+1)]).T
    return dt

def MRP(month, GR, SR, I_0, Lot, SS, LT):
  PR = [0]*month
  POR = [0]*month
  PAB = [0]*month
  PAB = initialize_PAB(SR,PR,GR,PAB,I_0)
  PR, POR, PAB = update_PR(SS,SR,PR,GR,POR,PAB,I_0,LT,Lot)
  result = dataframe_return(month, GR, SR, PAB, PR, POR)
  return result, POR

def MRP_level_wise(GR,month,*parameters):
  #level 단위 MRP 계산
  MRP_dict = {}
  POR_dict = {}
  BOM, I_0, LT, SS, Lot, SR = parameters
  for i in GR:
    name = i[0]
    temp_GR = i[1]
    temp_SR = SR[name]
    temp_I_0 = I_0[name]
    temp_Lot = Lot[name]
    temp_SS = SS[name]
    temp_LT = LT[name]
    print(name, temp_SR, temp_I_0, temp_Lot, temp_SS, temp_LT)
    MRP_dt, POR_dt = MRP(month, temp_GR, temp_SR, temp_I_0, temp_Lot, temp_SS, temp_LT)
    MRP_dict[name] = MRP_dt
    POR_dict[name]= POR_dt
  return MRP_dict, POR_dict  

In [7]:
# BOM '자기 이름', '이전 단계', '필요 개수'
BOM_jimmy = {'level0': ['BN-008',None,1],
       'level1': [['0810','BN-008',1],
                  ['0802','BN-008',1],
                  ['0807','BN-008',2],
                  ['0801','BN-008',1],
                  ['0808','BN-008',2]],
       'level2': [['0804','0807',1],['0803','0807',1],
                  ['0806','0808',1],['0805','0808',1]]
       }

I_0_jimmy = {'BN-008': 0,
      '0810':100,
      '0802':100,
      '0801':100,
      '0807':60,
      '0808':100,
      '0804':70,
      '0803':100,
      '0806':100,
      '0805':100
      }

LT_jimmy = {'BN-008': 0,
      '0810':2,
      '0802':1,
      '0801':1,
      '0807':1,
      '0808':1,
      '0804':1,
      '0803':1,
      '0806':1,
      '0805':1
      }


SS_jimmy = {'BN-008': 30,
      '0810':30,
      '0802':30,
      '0801':30,
      '0807':60,
      '0808':30,
      '0804':60,
      '0803':30,
      '0806':30,
      '0805':30
      }

LOT_jimmy = {'BN-008': 300,
      '0810':300,
      '0802':300,
      '0801':300,
      '0807':600,
      '0808':600,
      '0804':600,
      '0803':600,
      '0806':600,
      '0805':600
      }

SR_jimmy = {'BN-008': None,
      '0810':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0802':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0801':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0807':[600, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0808':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0804':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0803':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0806':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0805':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0]
      }

parameters = [BOM_jimmy,I_0_jimmy,LT_jimmy,SS_jimmy,LOT_jimmy,SR_jimmy]

In [8]:
month = 12
MPS = MPS_008['mps']

In [9]:
#level 1 GR 반환
GR_1 = MPS_to_GR(MPS,BOM_jimmy)
MRP_level_1, POR_level_1 = MRP_level_wise(GR_1,month,*parameters)
GR_2 = POR_to_GR(POR_level_1, BOM_jimmy, level = 2)
MRP_level_2, POR_level_2 = MRP_level_wise(GR_2,month,*parameters)

0810 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 300 30 2
0802 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 300 30 1
0807 [600, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 60 600 60 1
0801 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 300 30 1
0808 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1
0804 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 70 600 60 1
0803 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1
0806 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1
0805 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1


In [10]:
def display_MRP_levelwise(MRP):
  for i in MRP:
    print('*'*100)
    print(i)
    display(MRP[i])

In [11]:
display_MRP_levelwise(MRP_level_1)

****************************************************************************************************
0810


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,300,0,300,0,0,300,0,0,300,0,0,300
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,300,0,300,0,0,300,0,0,300,0,0,300
Planned order releases,300,0,0,300,0,0,300,0,0,300,300,0


****************************************************************************************************
0802


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,300,0,300,0,0,300,0,0,300,0,0,300
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,300,0,300,0,0,300,0,0,300,0,0,300
Planned order releases,0,300,0,0,300,0,0,300,0,0,300,300


****************************************************************************************************
0807


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,600,0,600,0,0,600,0,0,600,0,0,600
Scheduled receipts,600,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,60,60,60,60,60,60,60,60,60,60,60,60
Planned receipts,0,0,600,0,0,600,0,0,600,0,0,600
Planned order releases,0,600,0,0,600,0,0,600,0,0,600,0


****************************************************************************************************
0801


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,300,0,300,0,0,300,0,0,300,0,0,300
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,300,0,300,0,0,300,0,0,300,0,0,300
Planned order releases,0,300,0,0,300,0,0,300,0,0,300,300


****************************************************************************************************
0808


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,600,0,600,0,0,600,0,0,600,0,0,600
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,600,0,600,0,0,600,0,0,600,0,0,600
Planned order releases,0,600,0,0,600,0,0,600,0,0,600,600


In [12]:
display_MRP_levelwise(MRP_level_2)

****************************************************************************************************
0804


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,0
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,70,70,70,70,70,70,70,70,70,70,70,70
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,0
Planned order releases,600,0,0,600,0,0,600,0,0,600,0,0


****************************************************************************************************
0803


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,0
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,0
Planned order releases,600,0,0,600,0,0,600,0,0,600,0,0


****************************************************************************************************
0806


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,600
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,600
Planned order releases,600,0,0,600,0,0,600,0,0,600,600,0


****************************************************************************************************
0805


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,600
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,600
Planned order releases,600,0,0,600,0,0,600,0,0,600,600,0


# Actual Order 반영 이후

### SOP

In [13]:
def SOP(aom, sale_forecast, month, on_hand):
  #aom actual order의 마지막 month
  def inv_prev(i):
    try:
      return inv[i-1]
    except:
      return on_hand

  oper_plan_rd = [int(sum(sale_forecast)/month)]*month
  if aom>1:
    sale_forecast = [sale_forecast[i] - 10 if i >= aom else sale_forecast[i] for i in range(month)]
  else:
    sale_forecast = [sale_forecast[i] if i >= aom else sale_forecast[i] for i in range(month)]

  fix_plan_quan = int(sum(sale_forecast[aom:])/(month-aom))
  print(fix_plan_quan)
  oper_plan = [fix_plan_quan if i >= aom else oper_plan_rd[i] for i in range(month)]

  inv = []
  for i in range(month):
    invw = inv_prev(i) + oper_plan[i] - sale_forecast[i]
    inv.append(invw)

  df = pd.DataFrame({'Sales Forecast(in units)': sale_forecast, 'Actual_Order(in units)': order, 'Operations Plan(in units)': oper_plan, 'Inventory Plan(in units)': inv}, index=range(1, month + 1))  
  return {'oper_plan':oper_plan,'display': df.T}

In [14]:
sale_forecast = [70,80,120,130,60,80,90,100,110,80,130,150]
month = 12
on_hand = 0
order = [0]*month
actual_order = [50,60]
aom = len(actual_order)
order = [actual_order[i] if i<=aom-1 else order[i] for i in range(month)]

In [15]:
SOP_008 = SOP(aom, sale_forecast, month, on_hand)
SOP_008['display']

95


,1,2,3,4,5,6,7,8,9,10,11,12
Sales Forecast(in units),70,80,110,120,50,70,80,90,100,70,120,140
Actual_Order(in units),50,60,0,0,0,0,0,0,0,0,0,0
Operations Plan(in units),100,100,95,95,95,95,95,95,95,95,95,95
Inventory Plan(in units),30,50,35,10,55,80,95,100,95,120,95,50


###MPS

In [16]:
ss = 30 # safety stock
ls = 300 # lot size
month = 12
on_hand = 0 # on_hand
forecast = SOP_008['oper_plan'] # SOP의 operations plan
orders= [75,120,95,60,95,80,100,110,90,95,110,130]
pab = []
mps = [0]*month
atp = [0]*month

def pab_prev(i):
  try:
    return pab[i-1]
  except:
    return on_hand

def MPS(ss,ls,month,on_hand,forecast,orders,pab,mps,atp):

  for i in range(month):
    if pab_prev(i) - max(forecast[i],orders[i]) <= ss:
      mps[i] = ls
    pabw = pab_prev(i) + mps[i] - max(forecast[i],orders[i])
    pab.append(pabw)

  indices = [i for i, x in enumerate(mps) if x == ls]

  for i in range(len(indices)):
    atpm = ls
    if i==1: # on-hand 고려
      for j in range(0,indices[i]):
        atpm = atpm - orders[j]
      atpm = atpm + on_hand
      
      atp[0] = atpm
    else:
      for j in range(indices[i-1],indices[i]):
        atpm = atpm - orders[j]
      atp[indices[i-1]] = atpm

  atpm = ls
  atpm = atpm - orders[-1]
  atp[-1] = atpm

  df = pd.DataFrame({'Forecast': forecast, 'Orders': orders, 'Projected available balance': pab, 'Available-to-promise': atp, 'MPS': mps}, index=range(1, month + 1))
  return {'mps':mps,'display': df.T}

MPS_008 = MPS(ss,ls,month,on_hand,forecast,orders,pab,mps,atp)
MPS_008['display']

,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,100,100,95,95,95,95,95,95,95,95,95,95
Orders,75,120,95,60,95,80,100,110,90,95,110,130
Projected available balance,200,80,285,190,95,300,200,90,295,200,90,260
Available-to-promise,105,0,50,0,0,10,0,0,5,0,0,170
MPS,300,0,300,0,0,300,0,0,300,0,0,300


### MRP

In [17]:
def MPS_to_GR(MPS, BOM):
  level1 = BOM['level1']
  GR_1 = []
  for i in level1:
    GR_1.append([i[0],[j*i[2] for j in MPS]])
  return GR_1

def POR_to_GR(POR_dict, BOM, level = 2):
  #POR_dict = ['0810', [0, 600, ... POR 값들],]
  level = BOM[f'level{level}']
  GR_next = []
  for i in level:
    key = i[1]
    POR = POR_dict[key]
    GR_next.append([i[0],[j*i[2] for j in POR]])
  return GR_next

def initialize_PAB(SR,PR,GR,PAB,I_0):
  for i in range(len(PAB)):
    if i == 0:
      PAB[i] = I_0 + SR[i] + PR[i] - GR[i]
    else:
      PAB[i] = PAB[i-1] + SR[i] + PR[i] - GR[i]
  return PAB

def update_PR(SS,SR,PR,GR,POR,PAB,I_0,LT,Lot):
  for i in range(len(PAB)):
    if PAB[i]>=SS:
      pass
    else:
      PR[i]+=Lot
      POR[i-LT]+=Lot
      PAB = initialize_PAB(SR,PR,GR,PAB,I_0)
  return PR, POR, PAB

def dataframe_return(month, GR, SR, PAB, PR, POR):
    dt = pd.DataFrame({
        'Gross requirements': GR,
        'Scheduled receipts': SR,
        'Projected available balance': PAB,
        'Planned receipts': PR,
        'Planned order releases': POR
    },index = [i for i in range(1,month+1)]).T
    return dt

def MRP(month, GR, SR, I_0, Lot, SS, LT):
  PR = [0]*month
  POR = [0]*month
  PAB = [0]*month
  PAB = initialize_PAB(SR,PR,GR,PAB,I_0)
  PR, POR, PAB = update_PR(SS,SR,PR,GR,POR,PAB,I_0,LT,Lot)
  result = dataframe_return(month, GR, SR, PAB, PR, POR)
  return result, POR

def MRP_level_wise(GR,month,*parameters):
  #level 단위 MRP 계산
  MRP_dict = {}
  POR_dict = {}
  BOM, I_0, LT, SS, Lot, SR = parameters
  for i in GR:
    name = i[0]
    temp_GR = i[1]
    temp_SR = SR[name]
    temp_I_0 = I_0[name]
    temp_Lot = Lot[name]
    temp_SS = SS[name]
    temp_LT = LT[name]
    print(name, temp_SR, temp_I_0, temp_Lot, temp_SS, temp_LT)
    MRP_dt, POR_dt = MRP(month, temp_GR, temp_SR, temp_I_0, temp_Lot, temp_SS, temp_LT)
    MRP_dict[name] = MRP_dt
    POR_dict[name]= POR_dt
  return MRP_dict, POR_dict  

In [18]:
# BOM '자기 이름', '이전 단계', '필요 개수'
BOM_jimmy = {'level0': ['BN-008',None,1],
       'level1': [['0810','BN-008',1],
                  ['0802','BN-008',1],
                  ['0807','BN-008',2],
                  ['0801','BN-008',1],
                  ['0808','BN-008',2]],
       'level2': [['0804','0807',1],['0803','0807',1],
                  ['0806','0808',1],['0805','0808',1]]
       }

I_0_jimmy = {'BN-008': 0,
      '0810':100,
      '0802':100,
      '0801':100,
      '0807':60,
      '0808':100,
      '0804':70,
      '0803':100,
      '0806':100,
      '0805':100
      }

LT_jimmy = {'BN-008': 0,
      '0810':2,
      '0802':1,
      '0801':1,
      '0807':1,
      '0808':1,
      '0804':1,
      '0803':1,
      '0806':1,
      '0805':1
      }


SS_jimmy = {'BN-008': 30,
      '0810':30,
      '0802':30,
      '0801':30,
      '0807':60,
      '0808':30,
      '0804':60,
      '0803':30,
      '0806':30,
      '0805':30
      }

LOT_jimmy = {'BN-008': 300,
      '0810':300,
      '0802':300,
      '0801':300,
      '0807':600,
      '0808':600,
      '0804':600,
      '0803':600,
      '0806':600,
      '0805':600
      }

SR_jimmy = {'BN-008': None,
      '0810':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0802':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0801':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0807':[600, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0808':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0804':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0803':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0806':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0],
      '0805':[0, 0, 0, 0, 0, 0, 0 ,0, 0, 0, 0, 0]
      }

parameters = [BOM_jimmy,I_0_jimmy,LT_jimmy,SS_jimmy,LOT_jimmy,SR_jimmy]

In [19]:
month = 12
MPS = MPS_008['mps']

In [20]:
#level 1 GR 반환
GR_1 = MPS_to_GR(MPS,BOM_jimmy)
MRP_level_1, POR_level_1 = MRP_level_wise(GR_1,month,*parameters)
GR_2 = POR_to_GR(POR_level_1, BOM_jimmy, level = 2)
MRP_level_2, POR_level_2 = MRP_level_wise(GR_2,month,*parameters)

0810 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 300 30 2
0802 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 300 30 1
0807 [600, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 60 600 60 1
0801 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 300 30 1
0808 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1
0804 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 70 600 60 1
0803 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1
0806 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1
0805 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 100 600 30 1


In [21]:
def display_MRP_levelwise(MRP):
  for i in MRP:
    print('*'*100)
    print(i)
    display(MRP[i])

In [22]:
display_MRP_levelwise(MRP_level_1)

****************************************************************************************************
0810


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,300,0,300,0,0,300,0,0,300,0,0,300
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,300,0,300,0,0,300,0,0,300,0,0,300
Planned order releases,300,0,0,300,0,0,300,0,0,300,300,0


****************************************************************************************************
0802


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,300,0,300,0,0,300,0,0,300,0,0,300
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,300,0,300,0,0,300,0,0,300,0,0,300
Planned order releases,0,300,0,0,300,0,0,300,0,0,300,300


****************************************************************************************************
0807


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,600,0,600,0,0,600,0,0,600,0,0,600
Scheduled receipts,600,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,60,60,60,60,60,60,60,60,60,60,60,60
Planned receipts,0,0,600,0,0,600,0,0,600,0,0,600
Planned order releases,0,600,0,0,600,0,0,600,0,0,600,0


****************************************************************************************************
0801


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,300,0,300,0,0,300,0,0,300,0,0,300
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,300,0,300,0,0,300,0,0,300,0,0,300
Planned order releases,0,300,0,0,300,0,0,300,0,0,300,300


****************************************************************************************************
0808


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,600,0,600,0,0,600,0,0,600,0,0,600
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,600,0,600,0,0,600,0,0,600,0,0,600
Planned order releases,0,600,0,0,600,0,0,600,0,0,600,600


In [24]:
display_MRP_levelwise(MRP_level_2)

****************************************************************************************************
0804


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,0
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,70,70,70,70,70,70,70,70,70,70,70,70
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,0
Planned order releases,600,0,0,600,0,0,600,0,0,600,0,0


****************************************************************************************************
0803


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,0
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,0
Planned order releases,600,0,0,600,0,0,600,0,0,600,0,0


****************************************************************************************************
0806


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,600
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,600
Planned order releases,600,0,0,600,0,0,600,0,0,600,600,0


****************************************************************************************************
0805


,1,2,3,4,5,6,7,8,9,10,11,12
Gross requirements,0,600,0,0,600,0,0,600,0,0,600,600
Scheduled receipts,0,0,0,0,0,0,0,0,0,0,0,0
Projected available balance,100,100,100,100,100,100,100,100,100,100,100,100
Planned receipts,0,600,0,0,600,0,0,600,0,0,600,600
Planned order releases,600,0,0,600,0,0,600,0,0,600,600,0
